# Сбор данных с Web-scraping и API для социально-научных исследований
---
## Семинары 13-14. Введение в Selenium. Работа с открытыми судебными данными.

---
*ФСН, ОП "Политология", 2023-2024 гг.*

Лика Капустина,

lkapustina@hse.ru

**План занятия:**
1. [1. Selenium: введение и использование web-driver](#part1)
2. [2. Selenium: работа с элементами страницы](#part2)
---

**Основные ссылки:**
- [Документация Selenium на Python](https://selenium-python.readthedocs.io);
- [Неофициальный перевод документации Selenium на Python на русский](https://habr.com/ru/articles/248559/);
- [Документация BeautifulSoup на русском](https://www.crummy.com/software/BeautifulSoup/bs4/doc.ru/bs4ru.html);
- [Распространенные exceptions в Selenium](https://www.selenium.dev/selenium/docs/api/py/common/selenium.common.exceptions.html).

In [226]:
# импорт основных библиотек
import pandas as pd # работа с таблицами
import numpy as np  # математические операции и пр.
import tqdm         # для дальнейшего импорта прогресс-бара
import time         # для обычных пауз

## 1. Selenium: введение и использование web-driver<a name='part1'></a>

**Что такое Selenium?**
    
Selenium – программная библиотека для управления браузерами. Появилась в 2011 году и используется преимущественно для автоматизированного тестирования сайтов, но получила распространение и как инструмент, с помощью которого можно взаимодействовать с Javascript на сайтах и имитировать действия реального пользователя. 
    
**Ниже я продемонстрирую одну из причин, по которой вам иногда необходимо использовать Selenium**. 

Некоторые сайты не отдают html страниц в ответ на запросы ботов:

In [227]:
import requests # импортируем requests для отправки запросов
from bs4 import BeautifulSoup # импортируем BeautifulSoup

# получим html-разметку со страницы поиска Московского Городского Суда
html = requests.get('https://mos-gorsud.ru/search') # сайт московского городского суда
soup = BeautifulSoup(html.text) # получаем суп
print(soup.prettify()) # что тут? Да, тут пусто

Или в качестве `response` возвращают объекты с неполным содержанием. Например, сейчас мы попытаемся отправить запрос к странице [Высшей школы экономики в ВКонтакте](https://vk.com/hse). На этой странице содержится информация о группе Вышки в ВК, а также посты сообщества.

In [231]:
html = requests.get('https://vk.com/hse') # отправляем запрос
soup = BeautifulSoup(html.text) # обрабатываем html
#print(soup.prettify())

Зачастую единственное решение в таких случаях – либо пользоваться API сайта (если оно существует), либо использовать Selenium для имитации браузера и действий реального пользователя.

### Использование Selenium webdriver

**Что такое `webdriver`?** Webdriver – это основная сущность в Selenium, которая отвечает за управление браузером. 

Сейчас мы воспользуемся `WebdriverManager` – это класс, который поможет нам не мучаться с установкой WebDriver под каждую новую версию Google Chrome.

**Альтернатива**. Альтернативный способ установки webdriver для автоматизированной работы с браузером с помощью Selenium представляет из себя скачивание [ChromeDriver](https://chromedriver.chromium.org) под определенную версию Chrome. Это выглядело так: вы скачиваете Chromedriver под определенную версию своего браузера Chrome и каждый раз после выхода новой версии Google Chrome вы заново скачиваете подходящую версию ChromeDriver. С выхода версии 115 Google Chrome процесс скачивания подходящей версии стал [сложнее](https://www.browserstack.com/guide/run-selenium-tests-using-selenium-chromedriver). **Поэтому я советую вам пользоваться установкой Chromedriver через `ChromeDriverManager()`.**

In [2]:
!pip install selenium # установка selenium
!pip install webdriver-manager # установка пакета webdriver_manager

**`webdriver.Chrome()`, `ChromeDriverManager()`, `wb.get()`**

С помощью кода ниже мы cоздадим `selenium.webdriver.chrome.webdriver.WebDriver`, откроем определенную страницу, а по прошествии 5 секунд она будет закрыта. Такая запись с помощью `with` позволяет создать объект webdriver в первой строчке, а далее прописать код, который будет использован до автоматического прекращения работы webdriver (по окончанию последней строчки кода):

In [235]:
from selenium import webdriver # импорт драйвера
from selenium.webdriver.chrome.service import Service # импортируем сущность Service
from webdriver_manager.chrome import ChromeDriverManager # импорт драйвера для Google Chrome


with webdriver.Chrome(service=Service(ChromeDriverManager().install())) as wb:
    wb.get("https://www.hse.ru/edu/courses/875214615") # открываем определенную страницу
    time.sleep(5) # открываем страницу на 5 секунд, через 5 секунд она закроется

**Есть альтернатива**

Используйте строчку кода ниже, чтобы открыть окно Google Chrome с помощью webdriver и присвоить имя `wb` вашему `selenium.webdriver.chrome.webdriver.WebDriver`.

In [236]:
wb = webdriver.Chrome(service=Service(ChromeDriverManager().install())) # делается это так

In [237]:
wb # можем исследовать наш объект Webdriver

<selenium.webdriver.chrome.webdriver.WebDriver (session="7448954b20f5a3e6953d060ac24e8299")>

In [238]:
type(wb) # и посмотреть на его тип

selenium.webdriver.chrome.webdriver.WebDriver

**`selenium.webdriver.chrome.options`**

При использовании `Selenium` вы можете использовать большое количество разных возможностей: например, открыть окно браузера в полный экран. Для этого нужно поработать с классом `Options()`:

In [239]:
from selenium.webdriver.chrome.options import Options 

options = Options() # создаем сущность Options
options.add_argument("start-maximized") # уточняем, что хотим чтобы при открытии окно было открыто максимально широко
wb = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options) # создаем вебдрайвер
wb.get("https://www.hse.ru/edu/courses/875214615") # открываем ссылку

**Ручное взаимодействие с открытой страницей браузера и `wb.current_url`** 

С помощью метода `wb.current_url` вы можете получить ссылку на конкретную страницу. Зачем это нужно? В некоторых проектах вы можете переключаться между страницами (например, страницами новостей, профилями в социальных сетях) с помощью кнопок и других элементов (то есть, у вас не будет изначального списка ссылок, по которому вы будете идти). Чтобы сохранить в датафрейм информацию по данной конкретной странице, вам нужно будет использовать метод `wb.current_url`.

**Другая особенность Selenium – вы можете переключаться между разными страницами в открытом вами окне и браузер будет запоминать ваши действия**. Например:
* Вы можете открыть страницу социальной сети, залогиниться там вручную (не посылая значения в поля), а дальше использовать скрипт, написанный с использованием Selenium для выполнения автоматических действий. 
* Вы можете открыть страницу социальной сети, залогиниться там вручную (не посылая значения в поля), вручную ввести код подтверждения который пришел вам на телефон, а дальше использовать скрипт, написанный с использованием Selenium для выполнения автоматических действий.
* Вы можете открыть страницу и нажать на определенную кнопку настроек (например, *переключиться на светлую тему* или *скрыть уведомления*), и при работе с открытым в рамках данной сессии окном вебдрайвер запомнит настройки на этом сайте.

Продемонстрируем на примере, что браузер проследит за вашими действиями:

In [240]:
time.sleep(3)
print(wb.current_url) # это наша нынешняя ссылка

https://www.hse.ru/edu/courses/875214615


In [74]:
# давайте немного походим по сайту

In [241]:
# и теперь напечатаем ссылку страницы, которую мы только что открыли
time.sleep(3)
print(wb.current_url)

https://www.hse.ru/edu/courses/575402863


**`wb.close()` и `wb.quit()`**

Вы также можете закрыть открытое с помощью вебдрайвера окно. Это можно сделать с помощью метода `wb.close()` или `wb.quit()`.

* `wb.close()` - закрывает браузер, который сейчас открыт;

* `wb.quit()` - закрывает все окна и уничтожает сущность Webdriver. На практике вы не почувствуете разницы.

In [242]:
time.sleep(5) # сперва подождем пять секунд
wb.close()    # теперь закроем открытое окно

In [181]:
time.sleep(3) # подождем три секунды
wb.quit()     # другой способ

Обратите внимание - когда вы закрываете окно, вы больше **не можете взаимодействовать с тем окном, которым пользовались раньше**. Вам придется вновь создавать webdriver (`selenium.webdriver.chrome.webdriver.WebDriver`) и взаимодействовать уже с новым окном.

In [243]:
wb.current_url # ошибка

InvalidSessionIdException: Message: invalid session id
Stacktrace:
0   chromedriver                        0x0000000102fa12f8 chromedriver + 4625144
1   chromedriver                        0x0000000102f98ea3 chromedriver + 4591267
2   chromedriver                        0x0000000102b97c97 chromedriver + 392343
3   chromedriver                        0x0000000102bd5735 chromedriver + 644917
4   chromedriver                        0x0000000102c05754 chromedriver + 841556
5   chromedriver                        0x0000000102c004cf chromedriver + 820431
6   chromedriver                        0x0000000102bff6ae chromedriver + 816814
7   chromedriver                        0x0000000102b64caf chromedriver + 183471
8   chromedriver                        0x0000000102f61ac0 chromedriver + 4364992
9   chromedriver                        0x0000000102f66e86 chromedriver + 4386438
10  chromedriver                        0x0000000102f4672e chromedriver + 4253486
11  chromedriver                     

## 2. Selenium: работа с элементами страницы<a name='part2'></a>

### Selenium и работа с элементами страницы

**Другая причина по которой пользуются Selenium – это возможность взаимодействовать с элементами страницы: отправлять значения в формы, нажимать на кнопки и пр.** До некоторой информации невозможно добраться с помощью `requests` - например, если предварительно на сайте вам нужно нажать на кнопки вроде `Развернуть` и `Показать все`. С помощью Selenium вы можете взаимодействовать с этими самыми элементами.

**`wb.find_element(By.)`**

Что такое `.By`? `By` - класс, содержащий статические методы для идентификации элементов.

В свою очередь, `webelement` – это сущность, представляющая собой абстракцию над каким-то веб-элементом (кнопки, ссылки, поля ввода).

Вы можете использовать разные методы `.By` чтобы находить конкретные элементы:
<p></p>
<center>
    <table>
        <tr>
            <th><center>By.</center></th>
            <th><center>Описание</center></th>
        <tr><td>ID</td>
            <td>id конкретного элемента html</td></tr>
        <tr><td>TAG_NAME</td>
        <td>Название тега конкретного элемента html</td></tr>
        <tr><td>CLASS_NAME</td>
        <td>Название класса конкретного элемента html</td></tr>
        <tr><td>СSS_SELECTOR</td>
        <td>Локатор, который включает в себя конкретные уникальные атрибуты элемента html</td></tr>
        <tr><td>XPATH</td>
        <td>Путь по дереву html к элементу</td></tr>
        <tr><td>NAME</td>
            <td>Значение атрибута <code>name</code> конкретного элемента html</td></tr>
        <tr><td>PARTIAL_LINK_TEXT</td>
        <td>Возвращает элемент html, который содержит этот текст</td></tr>
        <tr><td>LINK_TEXT</td>
        <td>Возвращает элемент html, который содержит в точности этот текст</td></tr>
    </table>
</center>

**`wb.find_element(By.)` и `wb.find_elements(By.)`**: точно также, как при работе с объектами `bs4.BeautifulSoup` у вас есть возможность пользоваться двумя методами для поиска по элементам:
* Методом, возвращающим первый элемент, удовлетворяющий условию – `wb.find_element(By., ..)`
* Методом, возвращающим все элементы, удовлетворяющие условию – `wb.find_elements(By.)`.

Использование метода `wb.find_elements(By.)` удобно, когда вам нужно найти сразу несколько элементов, с которыми вы потом хотите взаимодействовать: например, если вы хотите найти несколько кнопок с одинаковым классом чтобы в дальнейшем в цикле прокликать их. 

Полная документация по поиску элементов с помощью Selenium [доступна по ссылке](https://selenium-python.readthedocs.io/locating-elements.html).

---

Давайте попробуем поискать элементы на странице поиска по судебным делам на сайте **[Московского городского суда](https://mos-gorsud.ru/search)**. Сайт Московского городского суда позволяет получить информацию о делах в самом Московском городском суде (суде субъекта федерации), так и о делах, которые рассматриваются в московских районных судах (наиболее распространенный уровень судов в России кроме мировых судей). Сперва откроем страницу:

In [244]:
wb = webdriver.Chrome(service=Service(ChromeDriverManager().install())) # создаем сущность вебдрайвера
url = 'https://mos-gorsud.ru/search' # сохраняем ссылку в переменную
wb.get(url) # открываем эту самую ссылку

Обратим внимание на элемент **<u>Нажмите, чтобы развернуть таблицу</u>**. 

Кликнем на него правой кнопкой мыши, нажмем `Просмотреть код`. 

Как мы видим, в html-разметке страницы этому элементу соответствует следующая строчка:

```<a href="#" class="js-table-modal-trigger">Нажмите, чтобы развернуть таблицу</a>```

**Как можно найти этот конкретный элемент на странице?** В случае с элементами, которые содержат в себе ссылки, можно использовать такие способы поиска:

In [246]:
from selenium.webdriver.common.by import By # импортируем класс .By
wb.find_element(By.LINK_TEXT, 'Нажмите, чтобы развернуть таблицу') # ищем элемент с ссылкой по конкретному значению

<selenium.webdriver.remote.webelement.WebElement (session="49f8502ee04e801b42cfcfc3bf5e230e", element="f.9F46158C5E34D443CF14D89564842CAA.d.7BC055606C330295AACBFD2FFABDD329.e.392")>

In [249]:
wb.find_element(By.PARTIAL_LINK_TEXT, 'Нажмите') # ищем элемент по частичному тексту ссылки

<selenium.webdriver.remote.webelement.WebElement (session="49f8502ee04e801b42cfcfc3bf5e230e", element="f.9F46158C5E34D443CF14D89564842CAA.d.7BC055606C330295AACBFD2FFABDD329.e.392")>

Можно искать элемент по названию класса:

In [250]:
wb.find_element(By.CLASS_NAME, 'js-table-modal-trigger') # нашли элемент по названию class

<selenium.webdriver.remote.webelement.WebElement (session="49f8502ee04e801b42cfcfc3bf5e230e", element="f.9F46158C5E34D443CF14D89564842CAA.d.7BC055606C330295AACBFD2FFABDD329.e.392")>

Но есть также и другие способы. Например, поиск по css-selector'у, xpath и пр. 

**Как его выполнить?**

Для этого сперва нужно будет обратиться к конкретному элементу, нажать на него правой кнопкой мыши, навести курсор на `Копировать`/`Copy`, и далее в выпадающем окне выбрать одно из значений. Например, при выборе `Копировать selector`/`Copy css-selector` вы получите css-selector: так вы сможете найти конкретный элемент, используя синтаксис css-selector'ов.

In [ ]:
#content > div > div.resultsearch_text > div > a

In [251]:
wb.find_element(By.CSS_SELECTOR, '#content > div > div.resultsearch_text > div > a') # скопировали css-selector

<selenium.webdriver.remote.webelement.WebElement (session="49f8502ee04e801b42cfcfc3bf5e230e", element="f.9F46158C5E34D443CF14D89564842CAA.d.7BC055606C330295AACBFD2FFABDD329.e.392")>

Другой способ - скопировать **xpath**. xpath – это путь к элементу html по дереву html документа. Чтобы получить xpath элемента, вам нужно навести на конкретный элемент, кликнуть правой кнопкой мыши, нажать `Копировать xpath` или `Копировать полную строку xpath`.

In [252]:
wb.find_element(By.XPATH, '//*[@id="content"]/div/div[1]/div/a') # просто xpath

<selenium.webdriver.remote.webelement.WebElement (session="49f8502ee04e801b42cfcfc3bf5e230e", element="f.9F46158C5E34D443CF14D89564842CAA.d.7BC055606C330295AACBFD2FFABDD329.e.392")>

In [253]:
wb.find_element(By.XPATH, '/html/body/div[2]/main/section/div[2]/div/div[1]/div/a') # полный xpath

<selenium.webdriver.remote.webelement.WebElement (session="49f8502ee04e801b42cfcfc3bf5e230e", element="f.9F46158C5E34D443CF14D89564842CAA.d.7BC055606C330295AACBFD2FFABDD329.e.392")>

**`.click()`**

Зачем мы ищем конкретные элемены? Правильно, чтобы с ними провзаимодействовать. Со многими элементами можно провзаимодействовать, просто кликнув на них. За это отвечает метод `.click()`. 

In [254]:
time.sleep(3) # делаем паузу
button_open_table = wb.find_element(By.CSS_SELECTOR, '#content > div > div.resultsearch_text > div > a') # элемент
button_open_table.click() # кликаем на конкретный элемент
time.sleep(3) # делаем паузу

**Поздравляю! Теперь вы научились находить элементы и кликать на них.**

<p></p>
<center><b><font size=4>Задача №1. Поиск элементов страницы. </font></b></center>

**На предыдущем шаге нам удалось открыть таблицу. Теперь напишите код который закроет эту таблицу, и используйте конструкцию `try-except`, чтобы этот код выглядел следующим образом:**

* Попытайтесь найти элемент по названию класса и кликнуть на него;
    * Если не получилось – попытайтесь найти элемент по css_selector и кликнуть на него;
        * Если не получилось - попытайтесь найти элемент по xpath и кликнуть на него;
            * Если не получилось - напечатайте строку `Увы, не получилось!` и кликнуть на него.
            
В каждом блоке кода после `except` добавьте паузу на 2 секунды.

In [1]:
# YOUR CODE HERE

### Selenium и отправка значений в поля

Кроме того, что с помощью Selenium вы можете искать конкретные значения и кликать на них, вы также можете работать с **полями для ввода информации**

Сперва кликнем на кнопку `Расширенная форма`, чтобы иметь возможности провзаимодействовать с полями для расширенного поиска по судебным делам.

In [76]:
# короче отправляем всякие параметры в поля запроса
url = 'https://mos-gorsud.ru/search' # сохраняем ссылку в переменную
wb.get(url) # открываем эту самую ссылку

In [257]:
time.sleep(3) # подождем три секунды
extended_search = wb.find_element(By.CLASS_NAME, 'extended ') # class этого элемента - "extended "
extended_search.click() # кликнем на элемент

Теперь наша задача - найти интересующие нас поля. Давайте найдем поле `Статья` и скопируем `css-selector` элемента, который соответствует этому полю на сайте:

In [259]:
article_field = wb.find_element(By.ID, 'codex')
article_field

<selenium.webdriver.remote.webelement.WebElement (session="49f8502ee04e801b42cfcfc3bf5e230e", element="f.9F46158C5E34D443CF14D89564842CAA.d.7BC055606C330295AACBFD2FFABDD329.e.37")>

**`.send_keys()`**

Теперь обратимся к методу `.send_keys()`. Этот метод работает так:
* В начале - название поля, в которое вы собираетесь отправить определенное значение;
* далее - название метода - `.send_keys()`;
* в скобках - значение, которое вы планируете отправить в определенную форму.

Давайте выполним поиск дел по статье **212 Уголовного кодекса Российской Федерации**: найдем дела о [массовых беспорядках](https://www.consultant.ru/document/cons_doc_LAW_10699/cdfbaa9aeaf8b47695af18e41433e4e3f5f4be5f/).

In [260]:
from selenium.webdriver.common.keys import Keys # посылка текста для ввода в формы (логина, пароля, поиска)

article = 'Ст. 212'
# мы обращаемся к конкретному элементу - article_field
article_field.send_keys(article + '\n') # в конце добавляем \n чтобы сразу запустить поиск
time.sleep(5) # делаем паузу

In [261]:
time.sleep(2)
wb.find_element(By.ID, 'case-index-search-form-btn').click()

**Поздравляю! Вы научились отправлять значения в поля**.

<p></p>
<center><b><font size=4>Задача №2. Работа с полями и элементами страницы</font></b></center>

**Используя расширенный поиск, вернитесь к главной странице, введите следующие значения в соответствующие поля и выполните поиск по сайту:**

* `Инстанция` – первая (подумайте, как можно выбрать это значение);
* `Год` – **2023**;
* В `Тексте` присутствует словосочетание **НИУ ВШЭ**.

**Покажите преподавательнице, что у вас открылось.**

In [262]:
url = 'https://mos-gorsud.ru/search'
wb.get(url)

# YOUR CODE HERE

Один из возможных вариантов решения:

In [263]:
# YOUR CODE HERE